In [115]:
import tweepy
from datetime import datetime

print_tweets = False
n_tweets = 10000
max_results = 100
    

auth = tweepy.OAuth1UserHandler(
    consumer_key, consumer_secret, access_token, access_token_secret
)

api = tweepy.API(auth)
client = tweepy.Client(bearer_token)
    
query = "#metoo"

tweet_fields = ['attachments', 'author_id', 'text', 'in_reply_to_user_id', 'referenced_tweets', 'entities']
next_token = None
end_time = datetime(year=2022,month=5,day=28)

all_tweets = []


# This endpoint/method returns Tweets from the last seven days
while len(all_tweets) < n_tweets:
    response = client.search_recent_tweets(query, tweet_fields=tweet_fields,
                                           max_results=max_results, next_token=next_token, end_time=end_time)
    tweets = response.data
    if tweets == None:
        break

    for tweet in tweets:
        all_tweets.append(tweet)
        
        if print_tweets:
            print(tweet.text)
            print("-------------------")
        
        if len(all_tweets) == n_tweets:
            break

    if print_tweets:
        print(len(all_tweets), "/", n_tweets)
    else:
        print("\r",len(all_tweets), "/", n_tweets, end="")

    if 'next_token' in response.meta:
        next_token = response.meta['next_token']
    else:
        break

 10000 / 10000

In [116]:
import json
with open("tweets.txt", "w+") as file:
    tweet_dicts = []
    for tweet in all_tweets:
        dict_tweet = dict(tweet)
        if tweet.referenced_tweets != None: 
            dict_tweet['referenced_tweets'] = [dict(r_tweet) for r_tweet in tweet.referenced_tweets]
        tweet_dicts.append(dict_tweet)
    file.write(json.dumps(tweet_dicts))

In [2]:
import json
with open("tweets.txt", "r") as file:
    all_tweets_json = json.load(file)

In [3]:
import pickle
from create_enron_network import tokenize_text, get_labels
from edgelabelgraph import EdgeLabelGraph

G = EdgeLabelGraph()

for tweet in all_tweets_json:
        
    author = tweet['author_id']
    
    # Use hashtags as labels
    if not ('entities' in tweet and 'hashtags' in tweet['entities']):
        continue
    labels = set([hashtag['tag'].lower() for hashtag in tweet['entities']['hashtags']])
    
    if 'mentions' in tweet['entities']:
        referenced_users = set([int(user['id']) for user in tweet['entities']['mentions']])
    
    if 'in_reply_to_user_id' in tweet:
        referenced_users.add(tweet['in_reply_to_user_id'])
    
    for user in referenced_users:
        if user != author:
            G.add_edge_with_labels((author, user), labels)
        
print(G.number_of_nodes())
print(G.number_of_edges())
print(G.density())

with open("tweets_graph.pkl", "wb") as file:
    pickle.dump(G, file)

8077
9429
1.1673888820106475
